In [ ]:
# hidden-but-run
import sys
sys.path.insert(0, "..")

In [ ]:
from elastipy import Search, query, connections
conn = connections.get()

In [ ]:
INDEX = "elastipy-example-setiquest-irc" 

In [ ]:
agg = Search(index=INDEX) \
    .agg_date_histogram("month", calendar_interval="month")
    
agg.execute().plot(to_index="month", kind="bar", figsize=(14, 4))

In [ ]:
agg = Search(index=INDEX) \
    .agg_date_histogram("month", calendar_interval="month") \
    .agg_terms("event", field="event")
    
df = agg.execute().df(to_index="month", flat="event", exclude="*.doc_count")
df.plot.bar(figsize=(14, 4), stacked=True)

In [ ]:
KEYWORDS = [
    "radio", "telescope", "meeting", "setiquest", 
    "form", "wiki", "community", "code", "project",
    "work", "help", "question", "alien", "ufo",
    "analysis", "source", "signal", "seti", "detect",
    "data", "thank"
]
agg = Search(index=INDEX) \
    .agg_filters("keyword", filters={
        key: query.QueryString(f"text: *{key}*")
        for key in KEYWORDS
    }) \
    .metric_cardinality("users", field="user")

agg.execute().dump.table(sort="-keyword.doc_count")

In [ ]:
agg = Search(index=INDEX) \
    .agg_filters("keyword", filters={
        key: query.QueryString(f"text: *{key}*")
        for key in KEYWORDS
    }) \
    .agg_date_histogram("year", fixed_interval="90d", min_doc_count=1)
agg2 = agg.metric_cardinality("users", field="user", return_self=True)

agg.execute().plot.heatmap(annot=True, fmt=".0f", figsize=(14, 7))
agg2.plot.heatmap(annot=True, fmt=".0f", figsize=(14, 7))

In [ ]:
s = Search(index=INDEX).match("text", "code")
print(list(s.agg_significant_terms(field="text", size=100).execute().keys()))

In [ ]:
s = Search(index=INDEX)#.query_string("text: *signal*")
s.agg_significant_terms(field="text", size=50).execute().dump.table(sort=-1)

In [ ]:
s = Search(index=INDEX)
agg = s.agg_terms(field="text")
agg.metric_cardinality(field="user")
agg.execute().plot.line("a0", "a1")

In [ ]:
Search(index=INDEX).execute().hits

In [ ]:
conn.termvectors(
    index=INDEX, 
    id="20100928-3",
    term_statistics=True,
    field_statistics=True,
)